# Test Stochastic Program implementation

In [1]:
import os
os.chdir('..') # move to the root directory (from dev)

In [2]:
import numpy as np
from utils import build_schema
from linmodel import LinProgModel
from citylearn.citylearn import CityLearnEnv

In [3]:
# set up parameters
dataset_dir = os.path.join('data','processed') # dataset directory

base_params = {
    'data_dir_path': os.path.join('data','processed'),
    'weather_data_path': 'weather.csv',
    'carbon_intensity_data_path': 'carbon_intensity.csv',
    'pricing_data_path': 'pricing.csv',
    'schema_name': 'schema_build_test',
    'battery_efficiencies': [0.95,0.95,0.95],
    'battery_energy_capacities': None,
    'battery_power_capacities': None,
    'pv_power_capacities': None
}

In [4]:
# set up LP costs
cost_dict = {
    'carbon': 1.0, #5e-1,
    'battery': 1e3, #1e3,
    'solar': 1e3, #2e3,
    'grid_capacity': 5e-2*365/0.95,
    'grid_excess': 10e-2*365/0.95,
    #'opex_factor': 20,
    'battery_power_ratio': 0.4
}

In [5]:
# set up load profiles
building_year_sets = [
    [(40,2015),(4,2012)],
    [(40,2016),(4,2013)],
    [(40,2014),(4,2015)],
]

In [6]:
# set up stochastic program
envs = []

for m, lys in enumerate(building_year_sets):
    # build schema
    params = base_params.copy()
    params['building_names'] = [f'TB{i}' for i in range(len(lys))]
    params['load_data_paths'] = [f'ly_{b}-{y}.csv' for b,y in lys]
    params['schema_name'] = f'SP_test_schema_{m}'
    schema_path = build_schema(**params)

    # Initialise CityLearn environment object.
    envs.append(CityLearnEnv(schema=schema_path))

    if m == 0: # initialise lp object
        lp = LinProgModel(env=envs[m])
    else:
        lp.add_env(env=envs[m])

In [7]:
sim_duration = 24*7*2 # 2 weeks of hours
cost_dict['opex_factor'] = 20*365*24/sim_duration # 20 year op. lifetime
lp.set_time_data_from_envs(t_start=3360,tau=sim_duration) # try summer time
lp.generate_LP(cost_dict,design=True)
lp.set_LP_parameters()

In [8]:
lp_results = lp.solve_LP(verbose=True,ignore_dpp=True)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) May 02 03:58:49 PM: Your problem has 7061 variables, 27 constraints, and 6054 parameters.
(CVXPY) May 02 03:58:49 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 02 03:58:49 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 02 03:58:49 PM: Compiling problem (target solver=SCIPY).
(CVXPY) May 02 03:58:49 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIPY
(CVXPY) May 02 03:58:49 PM: Applying reduction Dcp2Cone
(CVXPY) May 02 03:58:49 PM: Applying reduction CvxAttr2Constr


/Users/mal84/opt/miniconda3/envs/BD-VOI/lib/python3.8/site-packages/cvxpy/reductions/solvers/conic_solvers/scipy_conif.py:99: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (StrictVersion(scipy.__version__) < StrictVersion('1.6.1')):


8(2.73775e+06); Du: 0(2.45851e-07)
DuPh2       1764     2.9282488034e+06 Pr: 5812(4.6554e+06); Du: 0(2.91854e-07)
DuPh2       1996     3.0812833378e+06 Pr: 4331(6.30348e+06); Du: 0(5.25014e-07)
DuPh2       2331     3.6754461890e+06 Pr: 3493(646306); Du: 0(6.31604e-07)
DuPh2       2927     3.7987506042e+06 Pr: 4305(812500); Du: 0(9.86359e-07)
DuPh2       3352     4.0817425759e+06 Pr: 2562(1.00582e+06); Du: 0(8.95473e-07)
DuPh2       3697     4.2508303005e+06 Pr: 2098(165802); Du: 0(4.94187e-07)
DuPh2       4087     4.6731983688e+06 Pr: 1379(46631.3); Du: 0(5.61725e-07)
DuPh2       4478     4.9761069124e+06 Pr: 4790(2.19911e+06); Du: 0(4.84322e-07)
DuPh2       4814     5.1121371570e+06 Pr: 1418(128496); Du: 0(2.34111e-07)
DuPh2       5081     5.1742505756e+06 Pr: 2170(863034); Du: 0(2.75867e-07)
DuPh2       5481     5.2421777474e+06 Pr: 1619(268909); Du: 0(1.62783e-07)
DuPh2       5809     5.2715256299e+06 Pr: 263(8160.13); Du: 0(1.72414e-07)
DuPh2       6039     5.2804315094e+06 Pr: 0(2

In [9]:
print(lp_results['objective'])
print(lp_results['objective_contrs'])
print(lp_results['battery_capacities'])
print(lp_results['solar_capacities'])
print(lp_results['grid_capacity'])

5280427.674319058
[501297.20108079666, 118122.54436050732, 364470.6798504302, 1510126.6821715306, 1592636.521652997, 1193774.0452027956]
[[965.16717859]
 [627.46934307]]
[[505.72956195]
 [688.04448325]]
150.75762711864405


In [10]:
for m in range(len(envs)):
    print(np.max(lp_results['e_grids'][m]))

205.33589743589744
150.75762711864405
150.75762711864405
